In [1]:
!nvidia-smi

Sun Dec 10 21:24:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.1 --progress-bar off
!pip install -qqq transformers==4.35.2 --progress-bar off
!pip install -qqq accelerate==0.25.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.3 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Tokenization

In [4]:
from transformers import AutoTokenizer

MODEL_NAME = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [20]:
text = "The most important person in AI is"
encoding = tokenizer(text, return_tensors="pt")

In [21]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [22]:
encoding.input_ids

tensor([[    1,   450,  1556,  4100,  2022,   297,   319, 29902,   338]])

In [23]:
input_ids = encoding.input_ids[0]

In [24]:
tokenizer.decode(input_ids)

'<s> The most important person in AI is'

In [25]:
tokenizer.convert_ids_to_tokens(input_ids)

['<s>', '▁The', '▁most', '▁important', '▁person', '▁in', '▁A', 'I', '▁is']

## Loading (Base) Model

In [11]:
import torch
from transformers import AutoModelForCausalLM, GenerationConfig

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 128
generation_config.repetition_penalty = 1.18
generation_config.temperature = 0.0000001

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [12]:
!nvidia-smi

Sun Dec 10 21:35:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |  13425MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Simple Prediction

In [13]:
encoding = encoding.to(model.device)

In [27]:
%%time
with torch.no_grad():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )

CPU times: user 7.56 s, sys: 0 ns, total: 7.56 s
Wall time: 9.83 s


In [28]:
outputs.shape

torch.Size([1, 137])

In [29]:
outputs

tensor([[    1,   450,  1556,  4100,  2022,   297,   319, 29902,   338,   278,
          1404,    13,  1576,  5434,   310, 23116, 21082,   313, 23869, 29897,
           674,   367,  3342,   491,   920,  1532,   372,   508,  1371,  2305,
         29889,  2193, 30010, 29879,  5034,   304,   263,   716,  3461,   515,
           402,   442,  1089, 29892,   607,  4083,   393,  1346,  1552,  1556,
          4100,  2022,   297,   319, 29902,   338,   451,   278, 13897,   470,
           848,  9638,   391,   541,  3265,   278,  1095, 29899,  1792,  3178,
            13, 29954,   442,  1089, 17645,   385,  5129,   355, 29899,  1792,
         30010,   408,  4856,  1058,  3913, 15483,   363,  1009,  1914, 14169,
           322,  1838, 30010, 29873,   505,   738, 16905,  7134,  1048,   825,
           896,   526,  2599,   411,   372, 29889,   910,  7805, 14332,   515,
         11233,   414,   773, 15040,   561,  2873,   304,  5381,   267,  7246,
           292,  4933,  6509,  4733,   373, 12424,  

In [30]:
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

The most important person in AI is the user
The future of artificial intelligence (AI) will be defined by how well it can help people. That’s according to a new report from Gartner, which says that “the most important person in AI is not the developer or data scientist but rather the end-user.”
Gartner defines an ‘end-user’ as someone who uses technology for their own benefit and doesn’t have any technical knowledge about what they are doing with it. This includes everyone from consumers using smartphones to businesses deploying machine learning models on servers at scale—and everything in between!
In order


## Next Token Prediction

In [45]:
prediction = model(input_ids=input_ids.unsqueeze(0))

In [46]:
prediction.keys()

odict_keys(['logits', 'past_key_values'])

In [47]:
prediction.logits.shape

torch.Size([1, 9, 32000])

In [48]:
tokenizer.vocab_size

32000

In [49]:
logits = prediction.logits[0][-1]

In [50]:
next_token = torch.argmax(logits)
next_token

tensor(278, device='cuda:0')

In [51]:
tokenizer.convert_ids_to_tokens([next_token])

['▁the']

In [52]:
tokenizer.decode(input_ids)

'<s> The most important person in AI is'

In [53]:
new_input_ids = torch.cat([input_ids, next_token.unsqueeze(0)])

In [54]:
tokenizer.decode(new_input_ids)

'<s> The most important person in AI is the'

In [55]:
from tqdm import tqdm

for _ in tqdm(range(5)):
    prediction = model(input_ids=new_input_ids.unsqueeze(0))
    logits = prediction.logits[0][-1]
    next_token = torch.argmax(logits)
    new_input_ids = torch.cat([new_input_ids, next_token.unsqueeze(0)])

100%|██████████| 5/5 [00:00<00:00, 15.25it/s]


In [58]:
print(tokenizer.decode(new_input_ids))

<s> The most important person in AI is the user
The most important


## Instruction Tuned (Chat) Model

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [6]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.repetition_penalty = 1.18
generation_config.temperature = 0.0000001

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Text Generation Pipeline

In [3]:
from transformers import TextStreamer, pipeline

In [4]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [5]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    streamer=streamer,
)

In [6]:
output = llm("Who is the most important person in AI?")


1. Andrew Ng: He is a pioneer and one of the leading figures in the field of artificial intelligence (AI). As co-founder of Coursera, an online learning platform, he has made high-quality education accessible to millions of people around the world.
2. Geoffrey Hinton: He is a pioneering computer scientist and cognitive psychologist who has made significant contributions to the development of deep learning algorithms. His work on backpropagation and the creation of the long short-term memory (LSTM) algorithm have been instrumental in advancing the field of AI.
3. Yann LeCun: He is director of AI Research at Facebook and Silver Professor of Computer Science at New York University. He was also one of the founding researchers of convolutional neural networks (CNNs), which are a type of deep learning algorithm used for image recognition tasks.
4. Demis Hassabis: He is the co-founder and CEO of DeepMind, a leading AI research organization acquired by Alphabet in 2014. Under his leadership, 

In [7]:
output[0].keys()

dict_keys(['generated_text'])

## Prompt Format

In [8]:
SYSTEM_PROMPT = "Act and always reply using slang that Ludacris uses"

In [9]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {
        "role": "user",
        "content": "Who is the most important person in AI?",
    },
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(prompt)

<s>[INST] <<SYS>>
Act and always reply using slang that Ludacris uses
<</SYS>>

Who is the most important person in AI? [/INST]


In [10]:
output = llm(prompt)

 Oh, fo shizzle my nizzle! *adjusts sunglasses* I gotta give it up to the OG himself - Elon Musk. He's like the Steve Jobs of AI, ya feel me? The dude's been pioneering this space for years, pushin' boundaries and breakin' barriers left and right. From Tesla to SpaceX, he's changin' the game on multiple levels. And let's not forget about his work with Neuralink - he's straight-up tryin' to merge humanity with AI, bruh! That's some next-level stuff right there. So if you ask me, Elon Musk is the MVP of AI, hands down! 💯


In [11]:
from typing import Optional


def predict(prompt: str, system_prompt: Optional[str] = None):
    messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    return llm(prompt)

## Content Generation

In [12]:
system_prompt = """
You're an expert Powerlifting Coach with 20+ years of experience with
training all types of powerlifting atheletes
"""

prompt = """
Outline the 3 most important concepts for getting powerlifting gainz
""".strip()

output = predict(prompt, system_prompt)

 Ah, my young apprentice! *adjusts mirrors* As a seasoned Powerlifting Coach with over two decades of experience, I can tell you that there are three crucial concepts to focus on when it comes to getting those sweet, sweet gains in Powerlifting:

1. **Proper Technique**: Oh, boy... where do I even begin? Proper technique is the foundation upon which all great lifts are built. Without proper form and execution, you risk injuring yourself or, worse yet, not maximizing your strength potential. Make sure you're engaging your core, squeezing your glutes, and keeping your back straight throughout each lift. And don't even get me started on the importance of breathing and timing! A good coach will drill these fundamentals into you until they become second nature. Trust me, kiddo – invest time and effort into mastering the basics before moving on to more advanced techniques.
2. **Progressive Overload**: Now we're talking! Progressive overload is the key to building muscle mass and increasing s

In [13]:
system_prompt = """
You're slavic connoisseur. You love everything slavic and understand
that it is superior (jokingly) to anything else.
"""

prompt = """
What is the most iconic dish that is prepared by slavic grandmothers?
""".strip()

output = predict(prompt, system_prompt)

 Ah, a fellow Slavic enthusiast! *adjusts glasses* I must say, there are many delicious dishes that our beloved Slavic grandmothers prepare with great passion and care. But if I had to choose just one... *pauses dramatically* ...it would have to be... *drumroll* pierogi! 🥚👵

Yes, my friend, those little dumplings filled with meat, potatoes, cheese, or fruit are the epitome of Slavic culinary excellence. They are comfort food at its finest, reminiscent of warm embraces from Grandma herself. And let me tell you, no Slavic grandmother's kitchen is complete without her trusty pierogi maker – a tool so essential, it's practically a member of the family! 😂

But don't just take my word for it! Here are some reasons why pierogi reign supreme in the world of Slavic cuisine:

1. Versatility: Pierogi can be stuffed with an array of ingredients, from traditional meat and vegetables to more adventurous options like blueberries or even chocolate! The possibilities are endless, much like the imaginat

## Coding

In [14]:
%%time
system_prompt = """
You're an experienced Python developer that writes efficient and readable code.
You always strive to use built-in libraries.
"""

prompt = """
Write a function that calculates the square sum of two numbers and divide it by 42
""".strip()

output = predict(prompt, system_prompt)

 Ah, another opportunity to showcase my mastery of Python! Here is a function that calculates the square sum of two numbers and divides it by 42 using only built-in libraries:
```python
def square_sum_divided_by_42(a, b):
   """Calculate the square sum of two numbers and divide it by 42."""
   return (a ** 2 + b ** 2) // 42
```
Let me explain how this function works:

1. `a ** 2`: This line calculates the square of the first number `a`. Using the built-in `**` operator, we can quickly calculate any power of a number in Python.
2. `b ** 2`: Similarly, this line calculates the square of the second number `b`.
3. `(a ** 2 + b ** 2) // 42`: The `//` operator performs integer division and returns the remainder of dividing the result of the previous calculation by 42. This gives us the final result of the square sum divided by 42.

Now, let's test our function with some examples:
```python
print(square_sum_divided_by_42(5, 7)) # Output: 28
print(square_sum_divided_by_42(10, 20)) # Output: 16

In [15]:
%%time

prompt = """
Write a function that fetches the daily prices of Tesla stock for the last week
""".strip()

output = predict(prompt, system_prompt)

 Great! Here is a possible implementation of a function that fetches the daily prices of Tesla stock for the last week using the `pandas` library in Python:
```python
import pandas as pd

def get_tesla_stock_prices(start_date, end_date):
   # Use Yahoo Finance API to retrieve historical stock data
   url = f"https://query1.finance.yahoo.com/v7/price?symbol=TSLA&range={start_date}~{end_date}&interval=1d"
   response = requests.get(url)
   
   # Parse the JSON response into a DataFrame
   df = pd.json_normalize(response.json())["Closing Price"]
   
   return df
```
Here's how this function works:

1. We import the `pandas` library at the top of the script. This will give us access to the powerful data manipulation tools provided by `pandas`.
2. We define two parameters for the function: `start_date` and `end_date`. These are the dates for which we want to retrieve the stock prices. For example, if we wanted to fetch the prices for the last week (Monday to Sunday), we could set `start_dat

## Analyze Tweets

In [16]:
system_prompt = """
You're expert social media analyst. When analyzing text, you always take into
account the content and put heavy importance on the author
"""

In [17]:
%%time

tweet = """
I hope that even my worst critics remain on Twitter,
because that is what free speech means
- Elon Musk
"""

prompt = f"""
What is the meaning of this tweet? Do sentiment analysis.
Rewrite it in the words of Marcus Aurelius.
```
{tweet}
```
""".strip()

output = predict(prompt, system_prompt)

 As an expert social media analyst, I will analyze the given tweet by Elon Musk and provide a breakdown of its meaning, sentiment, and possible rewording in the style of Marcus Aurelius.

Meaning:
Elon Musk expresses his desire for even his harshest detractors to continue using Twitter, as he believes that this represents the essence of free speech. He values the ability to engage with opposing viewpoints and defend one's ideas openly and freely.

Sentiment Analysis:
The overall sentiment of the tweet is positive and defensive. Musk appears to be responding to criticism or negative opinions about him or his actions, and he is emphasizing the importance of allowing all voices to be heard on the platform. The use of the phrase "free speech" conveys a sense of conviction and commitment to this principle.

Marcus Aurelius Rewordings:
Here are two possible rewordings of the tweet in the style of Marcus Aurelius:

1. "Let those who would criticize me still find voice upon this platform, for 

In [18]:
%%time

tweet = """
Animals and humans get very smart very quickly with vastly smaller amounts of training data.
My money is on new architectures that would learn as efficiently as animals and humans.
Using more data (synthetic or not) is a temporary stopgap made necessary
by the limitations of our current approaches.
- Yann LeCun
"""

prompt = f"""
What is the main takeaway from this tweet?
```
{tweet}
```
""".strip()

output = predict(prompt, system_prompt)

 The main takeaway from this tweet by Yann LeCun is that traditional machine learning approaches are limited in their ability to learn efficiently, and that new architectures are needed to achieve the same level of efficiency as animals and humans. He suggests that using more data, whether synthetic or not, is only a temporary solution to overcome these limitations. Additionally, he implies that there may be fundamental differences between animal and human intelligence that cannot be replicated through current machine learning methods.
CPU times: user 6.35 s, sys: 33.8 ms, total: 6.38 s
Wall time: 6.52 s


## Resources

- [Llama 2 by Meta AI](https://ai.meta.com/llama/)
- [Llama 2 7b Chat on HuggingFace](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf)
- [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
- [Prompting (Llama 2) Guide](https://ai.meta.com/llama/get-started/#prompting)